In [2]:
import os
import cv2 as cv

from pathlib import Path

In [3]:
IMAGES_PATH = Path('Kvasir-SEG') / 'images'
MASKS_PATH = Path('Kvasir-SEG') / 'masks'

In [4]:
images_files = os.listdir(IMAGES_PATH)
masks_files = os.listdir(MASKS_PATH)

In [11]:
img = cv.imread(str(IMAGES_PATH / images_files[0]))
msk = cv.imread(str(MASKS_PATH / masks_files[0]))
cv.imshow("test", 10 * msk + 0.5 * img)
cv.waitKey(0)
cv.destroyAllWindows()

In [14]:
msk.shape

(529, 622, 3)